In [ ]:
## Reference
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [ ]:
### DATA DESCRIPTION

#     1. CRIM      per capita crime rate by town
#     2. ZN        proportion of residential land zoned for lots over 
#                  25,000 sq.ft.
#     3. INDUS     proportion of non-retail business acres per town
#     4. CHAS      Charles River dummy variable (= 1 if tract bounds 
#                  river; 0 otherwise)
#     5. NOX       nitric oxides concentration (parts per 10 million)
#     6. RM        average number of rooms per dwelling
#     7. AGE       proportion of owner-occupied units built prior to 1940
#     8. DIS       weighted distances to five Boston employment centres
#     9. RAD       index of accessibility to radial highways
#     10. TAX      full-value property-tax rate per $10,000
#     11. PTRATIO  pupil-teacher ratio by town
#     12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks 
#                  by town
#     13. LSTAT    % lower status of the population
#     14. MEDV     Median value of owner-occupied homes in $1000's

In [1]:
## LIBRARIES

import numpy as np
import pandas as pd
import math

In [2]:
## LOAD DATA
train = pd.read_csv('housing_train.txt', sep = "\s+",header=None)
#data.columns = ["a", "b", "c", "etc."]

#split into features and labels
train_x = train.iloc[:,0:13]
train_y = train.iloc[:,13]

test = pd.read_csv('housing_test.txt', sep = "\s+",header=None)

In [3]:
## TERMINAL NODES

# Create a terminal node value - Gets mean value of group
def to_terminal(group):
    return group.iloc[:,13].mean()


In [4]:
#BUILD TREE

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 0)
    return root

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
#     if not left or not right:
#         node['left'] = node['right'] = to_terminal(left + right)
#         return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['col']+1), node['val'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))


In [9]:
## SPLITS

# Split a dataset based on an column and an attribute value
# RETURNS 2 dataframes
def test_split(col, value, dataset):
    left = dataset.loc[dataset[col] < value]
    right = dataset.loc[dataset[col] >= value]
    return left, right

# Select the best split point for a dataset
def get_split(dataset):
    b_col, b_val, b_vari, b_groups = np.inf, np.inf, np.inf, None
    b_vari = np.var(dataset.iloc[:,13])
    #iterate rows
    for col in dataset.iloc[:,:-1]:
        colmax = round(dataset[col].max(),3)
        colmin = round(dataset[col].min(),3)
        test_values = np.linspace(colmin,colmax,20)
        
        for i in test_values:   
            groups = test_split(col, i, dataset)
            
            vari = 0
            for g in groups:
                #print("len: " + str(len(g)))
                vari += np.var(g.iloc[:,13]) * (len(g)/len(dataset))
            #print('X%d < %.3f Variance=%.3f' % ((col+1), dataset.iloc[index,col], vari))
               
            if vari < b_vari:
                b_vari = vari
                b_col = col
                b_groups = groups
                b_val = i
                
    return {'col':b_col, 'val':b_val, 'groups':b_groups}

In [6]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['col']] < node['val']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [7]:
# Classification and Regression Tree Algorithm returns MSE
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for i in range(len(test[0])):
        prediction = predict(tree, test.iloc[i,:])
        predictions.append(prediction)
    mse = 0
    for i in range(len(predictions)):
        mse += ((predictions[i] - test.iloc[i,13]) ** 2) / 433
    return(mse, tree)

In [10]:
tree_depths = [3,6,9,12,15,18]
for j in tree_depths:
    print("Tree Depth: " + str(j))
    train_error, train_tree = decision_tree(train, train, j, 8)
    print('Train Error: ' + str(train_error))
    #TEST ERROR
    predictions = list()
    for i in range(len(test[0])):
        prediction = predict(train_tree, test.iloc[i,:])
        predictions.append(prediction)
        mse = 0
        for i in range(len(predictions)):
            mse += ((predictions[i] - test.iloc[i,13]) ** 2) / len(predictions)
    print('Test Error: ' + str(mse))

Tree Depth: 3
Train Error: 14.380032646726978
Test Error: 23.125225336875257
Tree Depth: 6
Train Error: 6.712191923096406
Test Error: 17.33541176521059
Tree Depth: 9
Train Error: 4.304423529956529
Test Error: 19.618981170376376
Tree Depth: 12
Train Error: 3.4560016811329484
Test Error: 20.312433724825823
Tree Depth: 15
Train Error: 3.313946937204444
Test Error: 19.832812160821533
Tree Depth: 18
Train Error: 3.2986698009457838
Test Error: 20.086621282443158


In [11]:
tree_depths = [3,4,5,6,7,8,9]
for j in tree_depths:
    print("Tree Depth: " + str(j))
    train_error, train_tree = decision_tree(train, train, j, 8)
    print('Train Error: ' + str(train_error))
    #TEST ERROR
    predictions = list()
    for i in range(len(test[0])):
        prediction = predict(train_tree, test.iloc[i,:])
        predictions.append(prediction)
        mse = 0
        for i in range(len(predictions)):
            mse += ((predictions[i] - test.iloc[i,13]) ** 2) / len(predictions)
    print('Test Error: ' + str(mse))

Tree Depth: 3
Train Error: 14.380032646726978
Test Error: 23.125225336875257
Tree Depth: 4
Train Error: 9.667072556113034
Test Error: 19.577508279780787
Tree Depth: 5
Train Error: 8.03007393627163
Test Error: 17.887270403722635
Tree Depth: 6
Train Error: 6.712191923096406
Test Error: 17.33541176521059
Tree Depth: 7
Train Error: 5.542935093768232
Test Error: 18.26105675180934
Tree Depth: 8
Train Error: 4.913438257115118
Test Error: 18.396344532282598
Tree Depth: 9
Train Error: 4.304423529956529
Test Error: 19.618981170376376


In [ ]:
train_error, train_tree = decision_tree(train, train, 15, 8)

In [ ]:
train_error

In [ ]:
#TEST ERROR
predictions = list()
for i in range(len(test[0])):
    prediction = predict(train_tree, test.iloc[i,:])
    predictions.append(prediction)
    mse = 0
    for i in range(len(predictions)):
        mse += ((predictions[i] - test.iloc[i,13]) ** 2) / len(predictions)

In [ ]:
mse

In [ ]:
train_tree